In [1]:
import os
import cv2
import skimage.io
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import gc
from sklearn.model_selection import train_test_split

In [2]:
# Make csv
# Save images in a folder 
# Add labels to csv

In [3]:
train_df = pd.read_csv('/kaggle/input/prostate-cancer-grade-assessment/train.csv')
radboud_csv = train_df[train_df['data_provider'] == 'radboud']
karolinska_csv = train_df[train_df['data_provider'] != 'radboud']
r_train, r_test = train_test_split(
    radboud_csv,
    test_size=0.2, random_state=42
)

k_train, k_test = train_test_split(
    karolinska_csv,
    test_size=0.2, random_state=42
)
train_df = pd.concat([r_train, k_train])
test_df = pd.concat([r_test, k_test])

print(train_df.shape)
print(test_df.shape)

(8492, 4)
(2124, 4)


In [4]:
TRAIN = '../input/panda-16x128x128-tiles-data/train'
MASKS = '../input/panda-16x128x128-tiles-data/masks'

In [5]:
def get_dominant_color(a):
    colors, count = np.unique(a.reshape(-1,a.shape[-1]), axis=0, return_counts=True)
    return colors[count.argmax()]

In [6]:
def getLabel(row):
    if(row[1].get(3) == "0+0"):
        return True, "0"
    if(row[1].get(3) == "3+3"):
        return True, "3"
    if(row[1].get(3) == "4+4"):
        return True, "4"
    if(row[1].get(3) == "5+5"):
        return True, "5"
    return False, "-1"
        

In [7]:
from PIL import Image, ImageFilter

import PIL
import random

images = []
labels = []
import csv
base_dir = "/kaggle/working/imgDir/"
with open('train.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Id", "Label"])
    num = 0
    writer = csv.writer(file)
    for row,z in zip(train_df.iterrows(),tqdm(range(len(train_df)))):
        b, l = getLabel(row)
        if(b):
            score = l
            name = row[1].get(0)
            for i in range(6):
                if(not os.path.isfile(MASKS+'/'+name+"_"+str(i)+".png")):
                    break

                mask = Image.open(MASKS+'/'+name+"_"+str(i)+".png")


                im = Image.open(TRAIN+'/'+name+"_"+str(i)+".png")

                dis = get_dominant_color(np.asarray(mask))
                angle = random.randint(0,360)

                if((3 in dis)|(4 in dis)|(5 in dis)):
                    im.save(base_dir+str(num)+".png","PNG")
                    writer.writerow([num, score])
                    num = num + 1
                    im.rotate(angle).save(base_dir+str(num)+".png","PNG")
                    writer.writerow([num, score])
                    num = num + 1
                    im.filter(ImageFilter.GaussianBlur).save(base_dir+str(num)+".png","PNG")
                    writer.writerow([num, score])
                    num = num + 1
                    im.filter(ImageFilter.SHARPEN).save(base_dir+str(num)+".png","PNG")
                    writer.writerow([num, score])
                    num = num + 1
                    im.filter(ImageFilter.SMOOTH).save(base_dir+str(num)+".png","PNG")
                    writer.writerow([num, score])
                    num = num + 1
                    im.transpose(PIL.Image.FLIP_LEFT_RIGHT).save(base_dir+str(num)+".png","PNG")
                    writer.writerow([num, score])
                    num = num + 1
                    im.transpose(PIL.Image.FLIP_TOP_BOTTOM).save(base_dir+str(num)+".png","PNG")
                    writer.writerow([num, score])
                    num = num + 1

                else:
                    im.save(base_dir+str(num)+".png","PNG")
                    writer.writerow([num, 0])
                    num = num + 1
                    im.rotate(angle).save(base_dir+str(num)+".png","PNG")
                    writer.writerow([num, 0])
                    num = num + 1
                    im.filter(ImageFilter.GaussianBlur).save(base_dir+str(num)+".png","PNG")
                    writer.writerow([num, 0])
                    num = num + 1
                    im.filter(ImageFilter.SHARPEN).save(base_dir+str(num)+".png","PNG")
                    writer.writerow([num, 0])
                    num = num + 1
                    im.filter(ImageFilter.SMOOTH).save(base_dir+str(num)+".png","PNG")
                    writer.writerow([num, 0])
                    num = num + 1
                    im.transpose(PIL.Image.FLIP_LEFT_RIGHT).save(base_dir+str(num)+".png","PNG")
                    writer.writerow([num, 0])
                    num = num + 1
                    im.transpose(PIL.Image.FLIP_TOP_BOTTOM).save(base_dir+str(num)+".png","PNG")
                    writer.writerow([num, 0])
                    num = num + 1


                x_shift = random.randint(1,1000) # +left/-right 
                y_shift = random.randint(1,1000) # +up/-down 
                translate = mask.transform(mask.size, Image.AFFINE, (1, 0, x_shift, 0, 1, y_shift))
                disTrans = get_dominant_color(np.asarray(translate))

                imT = im.transform(im.size, Image.AFFINE, (1, 0, x_shift, 0, 1, y_shift))
                imT.save(str(num)+".png","PNG")
                
                if((3 in disTrans)|(4 in disTrans)|(5 in disTrans)):
                    writer.writerow([num, score])
                else:
                    writer.writerow([num, 0])
                num = num + 1

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/imgDir/0.png'

In [8]:
mkdir "/kaggle/working/testImgDir/"

In [9]:
from PIL import Image, ImageFilter
import random
test_dir = "/kaggle/working/testImgDir/"


with open('test.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Id", "Label"])
    num = 0
    writer = csv.writer(file)
    for row,z in zip(test_df.iterrows(),tqdm(range(len(test_df)))):
        b, l = getLabel(row)
        if(b):
            score = l
            name = row[1].get(0)
            for i in range(6):
                if(not os.path.isfile(MASKS+'/'+name+"_"+str(i)+".png")):
                    break

                mask = Image.open(MASKS+'/'+name+"_"+str(i)+".png")
                dis = get_dominant_color(np.asarray(mask))

                im = Image.open(TRAIN+'/'+name+"_"+str(i)+".png")
                im.save(test_dir+str(num)+".png","PNG")
                
                if((3 in dis)|(4 in dis)|(5 in dis)):
                    writer.writerow([num, score])
                else:
                    writer.writerow([num, 0])
                num = num + 1